In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib
import os
import numpy as np
from shapely.geometry import Point
import matplotlib.pyplot as plt

c:\Users\hp\anaconda3\envs\mm\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


# Import des données

### Limites admin

In [2]:
path_data = r"C:\Users\hp\Documents\MITSIOMOTU\Prise phase2\2. Data\1. Analyse cas d'usage\MMRE"

# Import des limites administratives
cantons = gpd.read_file(os.path.join(path_data,'data','limites admin','14_11_22_Togo_Cantons2_4326.gpkg'))
communes = gpd.read_file(os.path.join(path_data,'data','limites admin','14_11_22_Communes_du_Togo_2.gpkg'))

cantons = cantons.to_crs(epsg='25231')
communes = communes.to_crs(epsg = '25231')

reg = 'Maritime'

### Réseau BT

In [3]:
### Import du réseau BT
# Réseaux et branchements BT
support_bt_pays = gpd.read_file(os.path.join(path_data,'data','réseau BT','BT_Support_20240102.shp'))

ligne_bt_pays = gpd.read_file(os.path.join(path_data,'data','réseau BT','BT_Troncon_20240102.shp'))
ligne_bt_GL = gpd.read_file(os.path.join(path_data,'data','réseau BT','bt_corriges_par_rapport_aux_branchements_manquants.gpkg'))
ligne_bt_manquant = gpd.read_file(os.path.join(path_data,'data','réseau BT','Reseau_BT_manquant.shp'))

# Fusionner les GeoDataFrame
ligne_agg = pd.concat([ ligne_bt_pays[['longueur2d',"geometry"]] , 
                              ligne_bt_GL[['Longueur',"geometry"]] , 
                              ligne_bt_manquant[['Longueur',"geometry"]]], ignore_index=True).reset_index()

ligne_agg = ligne_agg.to_crs(epsg='25231')
support_bt_pays = support_bt_pays.to_crs(epsg='25231')


poteaux_cantons = gpd.sjoin_nearest(support_bt_pays,cantons,how='inner',max_distance=5000)
ligne_cantons = gpd.sjoin_nearest(ligne_agg,cantons,how='inner',max_distance=5000)

poteaux_gdf = poteaux_cantons[poteaux_cantons.geometry.notnull()]
lignes_bt_gdf = ligne_cantons[ligne_cantons.geometry.notnull()]
lignes_bt_gdf = lignes_bt_gdf [lignes_bt_gdf.geometry.length  <= 5000 ]

path_out = os.path.join(path_data,'data','réseau BT')
lignes_bt_gdf.to_file(os.path.join(path_out, 'lignes_bt_total.gpkg') , driver = 'GPKG')
poteaux_gdf.to_file(os.path.join(path_out, 'poteaux_bt_total.gpkg') , driver = 'GPKG')



c:\Users\hp\anaconda3\envs\mm\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\hp\anaconda3\envs\mm\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


### Branchements / Connexions CEET

In [ ]:
### Import des branchements
branchements_pays = gpd.read_file(os.path.join(path_data,'data','branchement','branchement_copie.gpkg'))
branchements_GL = gpd.read_file(os.path.join(path_data,'data','branchement','Branchements_manquants_Golfe_Agoe.shp'))
branchements_manquant = gpd.read_file(os.path.join(path_data,'data','branchement','Branchements_manquants.shp'))

# Convertir les lignes en points en prenant leur centroïde
branchements_pays["geometry"] = branchements_pays["geometry"].centroid

# Fusionner les GeoDataFrame
branchements_agg = pd.concat([ branchements_pays[['longueur2d',"geometry"]] , 
                              branchements_GL[['ID2',"geometry"]] , 
                              branchements_manquant[['ID2',"geometry"]]], ignore_index=True).reset_index()

branchements_agg = branchements_agg.to_crs(epsg='25231')

branchements_canton = gpd.sjoin_nearest(branchements_agg,cantons,how='inner',max_distance=5000)

# Filtrer les géométries nulles
branchements_gdf = branchements_canton[branchements_canton.geometry.notnull()]
# Suppression des linéaires avec une longueur supérieure à 1000 mètres
branchements_gdf = branchements_gdf[(branchements_gdf["longueur2d"] <= 1000) | (branchements_gdf["longueur2d"].isna())]
branchements_gdf = branchements_gdf.drop(columns='index_right')

path_out = os.path.join(path_data,'data','branchement')
branchements_gdf.to_file(os.path.join(path_out, 'branchement_total.gpkg') , driver = 'GPKG')


### Ménages

In [ ]:
### Import des ménages
# Import des données des ménages
menage_centrale = pd.read_csv(os.path.join(path_data,'data','ménage','Menages_centrale.csv'))
menage_kara = pd.read_csv(os.path.join(path_data,'data','ménage','Menages_kara.csv'))
menage_dagl = pd.read_csv(os.path.join(path_data,'data','ménage','Menages_DAGL.csv'))
menage_dagl_suite = pd.read_csv(os.path.join(path_data,'data','ménage','Menages_DAGL_suite.csv'))

menage_agg = pd.concat([menage_centrale , menage_kara,menage_dagl,menage_dagl_suite]).reset_index()

# Créer des objets géométriques Point à partir des colonnes LONGITUDE et LATITUDE
geometry = [Point(xy) for xy in zip(menage_agg['LONGITUDE'], menage_agg['LATITUDE'])]

# Créer un GeoDataFrame avec le système de coordonnées initial (WGS84, EPSG:4326)
menages_gdf = gpd.GeoDataFrame(menage_agg, geometry=geometry, crs="EPSG:4326")

# Reprojeter le GeoDataFrame en EPSG:25231
menages_gdf = menages_gdf.to_crs(epsg=25231)

menages_canton = gpd.sjoin_nearest(menages_gdf,cantons,how='inner',max_distance=5000)

menage_gdf = menages_canton[menages_canton.geometry.notnull()]
menage_gdf = menage_gdf.drop(columns='index_right')
menage_gdf2 = gpd.GeoDataFrame(menage_gdf, geometry='geometry', crs=menage_gdf.crs).reset_index(drop=True)

path_out = os.path.join(path_data,'data','ménage')
menage_gdf2.to_file(os.path.join(path_out, 'menage_total.gpkg') , driver = 'GPKG')

### Concessions

In [ ]:
### Concessions
# Import des données de concessions
concession_centrale = gpd.read_file(os.path.join(path_data,'data','concession','RGPH5_CARTO_Centrale_Concessions1.gpkg'))
concession_kara = gpd.read_file(os.path.join(path_data,'data','concession','RGPH5_CARTO_Kara_Concessions1.gpkg'))
concession_plateaux= gpd.read_file(os.path.join(path_data,'data','concession','RGPH5_CARTO_Plateaux_Concessions1.gpkg'))
concession_savanes = gpd.read_file(os.path.join(path_data,'data','concession','RGPH5_CARTO_Savanes_Concessions1.gpkg'))
concession_maritime = gpd.read_file(os.path.join(path_data,'data','concession','RGPH5_CARTO_Maritimes_Concessions1.gpkg'))
concession_dagl = gpd.read_file(os.path.join(path_data,'data','concession','RGPH5_CARTO_DAGL_Concessions1.gpkg'))


concession = pd.concat([concession_centrale,concession_kara,concession_plateaux,concession_savanes, concession_maritime,concession_dagl]).reset_index()
concession = concession.to_crs(epsg='25231')

concessions_canton = gpd.sjoin_nearest(concession,cantons,how='inner',max_distance=5000)

concession_gdf = concessions_canton[concessions_canton.geometry.notnull()]
concession_gdf = concession_gdf.drop(columns='index_right')


In [ ]:
# Créer des géométries pour les concessions associées aux ménages
menage_gdf["geometry_concession"] = menage_gdf.apply(
    lambda row: Point(row["LONGITUDE_CONCESSION"], row["LATITUDE_CONCESSION"]),
    axis=1,
)

# Convertir les géométries en GeoDataFrame
concessions_menage_gdf= gpd.GeoDataFrame(menage_gdf[['ID_HHFtth','TAILLE_MENAGE','geometry_concession']], geometry="geometry_concession", crs="EPSG:4326")
concessions_menage_gdf = concessions_menage_gdf.to_crs(epsg = '25231')

# Appliquer un buffer de 1 mm 
buffer_size = 0.001  # En mètres 

# Effectuer la jointure spatiale (concessions les plus proches pour chaque ménage)
joined = gpd.sjoin_nearest(
    concessions_menage_gdf,
    concession_gdf,
    how="left",
    max_distance=buffer_size,
    distance_col="distance_to_concession"
)

# Identifier les ménages sans concessions associées dans la tolérance du buffer
gdf_concessions_manquantes = joined[joined["distance_to_concession"].isna()]

col_concess_manquante = ['ID_HHFtth','TAILLE_MENAGE','geometry_concession']

gdf_concessions_missing = gpd.sjoin(gdf_concessions_manquantes[col_concess_manquante],cantons, how='inner',predicate="intersects")

gdf_concessions_missing = gdf_concessions_missing.rename(columns={'geometry_concession': 'geometry'})

# Sauvegarde les concessions manquantes issues des fichiers de ménages
path_out = os.path.join(path_data,'data','concession','concession_manquante')
gdf_concessions_missing.to_file(os.path.join(path_out,'concessions_manquantes.gpkg'), driver="GPKG")

## Concaténation avec les concessions initiales
concessions_total = pd.concat([concession_gdf, gdf_concessions_missing], ignore_index=True)

concessions_total = concessions_total.drop(columns=['index_left', 'index_right'], errors='ignore')
concessions_total = gpd.GeoDataFrame(concessions_total, geometry='geometry', crs=concession_gdf.crs).reset_index(drop=True)

# Sauvegarde du fichier concessions totales
path_out= os.path.join(path_data,'data','concession')
concessions_total.to_file(os.path.join(path_out,'concessions_total.gpkg'),driver = 'GPKG')

### Générations des buffers

In [ ]:
def generate_and_merge_buffers(poteaux_gdf, lignes_gdf, cantons,distance):
    """
    Crée un buffer autour des poteaux et des lignes, puis fusionne les buffers par canton.
    """
    # Générer les buffers
    poteaux_gdf["buffer"] = poteaux_gdf.geometry.buffer(distance)
    lignes_gdf["buffer"] = lignes_gdf.geometry.buffer(distance)

    # Concaténer poteaux et lignes
    all_buffers = gpd.GeoDataFrame(
        pd.concat([poteaux_gdf[["canton_nom", "buffer"]], lignes_gdf[["canton_nom", "buffer"]]]),
        crs=poteaux_gdf.crs).set_geometry("buffer") 

    # Dissolve pour fusionner par canton
    merged_buffers = all_buffers.dissolve(by="canton_nom")

    # Nettoyage : renommer la géométrie pour éviter les erreurs
    merged_buffers = merged_buffers.rename(columns={"buffer": "geometry"}).set_geometry("geometry")

    # Ajouter les informations administratives
    merged_buffers = merged_buffers.merge(
        cantons[["canton_nom", "region_nom", "prefecture_nom", "commune_nom"]],
        on="canton_nom",
        how="left"
    )

    return merged_buffers

# Définition des distances
distances = [60, 70, 80]
path_out = os.path.join(path_data,'data','réseau BT','buffer')
# Variables pour stocker les buffers pour chaque distance
buffer_60 = None
buffer_70 = None
buffer_80 = None

# Génération des buffers et sauvegarde des fichiers
for distance in distances:
    buffer_gdf = generate_and_merge_buffers(poteaux_gdf, lignes_bt_gdf,cantons, distance)

    # Assigner le buffer généré à la variable correspondante
    if distance == 60:
        buffer_60 = buffer_gdf
    elif distance == 70:
        buffer_70 = buffer_gdf
    elif distance == 80:
        buffer_80 = buffer_gdf
        
    # Définir le chemin du fichier
    output_file = os.path.join(path_out, f"zones_tampon_{distance}m.gpkg")

    # Sauvegarder le fichier GeoPackage
    buffer_gdf.to_file(output_file, driver="GPKG", layer=f"buffers_{distance}m")

    print(f"Fichier enregistré : {output_file}")